In [15]:
!pip install torch torchvision pillow --quiet

In [1]:
from torchvision.models import EfficientNet_V2_S_Weights
import torchvision.models as models

model = models.efficientnet_v2_s(EfficientNet_V2_S_Weights.DEFAULT)
model

/home/charecktowa/Documentos/CIC/Taller/.env/lib/python3.12/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /home/charecktowa/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100.0%


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [2]:
from torchvision import transforms

model.eval()

preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],  # Valores promedio de ImageNet
            std=[0.229, 0.224, 0.225],  # Desviación estándar de ImageNet
        ),
    ]
)

In [3]:
import urllib.request

# Descargar las etiquetas de ImageNet
url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
urllib.request.urlretrieve(url, "imagenet_classes.txt")

('imagenet_classes.txt', <http.client.HTTPMessage at 0x78ef597a98e0>)

In [4]:
# Cargar las etiquetas de las clases
with open("imagenet_classes.txt") as f:
    labels = [line.strip() for line in f.readlines()]

labels[:5]

['tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead']

In [5]:
import torch

from PIL import Image

# Función para hacer predicciones
def predict(image_path):
    # Cargar y transformar la imagen
    input_image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)  # Crear un batch con la imagen

    # Verificar si hay GPU disponible
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando {device}")
    model.to(device)
    input_batch = input_batch.to(device)

    # Realizar la predicción sin calcular gradientes
    with torch.no_grad():
        output = model(input_batch)

    # Aplicar softmax para obtener probabilidades
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # Obtener las 5 mejores predicciones
    top5_prob, top5_catid = torch.topk(probabilities, 5)

    # Mostrar los resultados
    for i in range(top5_prob.size(0)):
        print(f"{labels[top5_catid[i]]}: {top5_prob[i].item()*100:.2f}%")

In [8]:
image = "https://raw.githubusercontent.com/pytorch/hub/master/images/dog.jpg"

# Descargar la imagen
urllib.request.urlretrieve(image, "dog.jpg")

# Hacer la predicción
image = "./bottle.jpeg"

predict(image)

Usando cuda
water bottle: 80.91%
pop bottle: 0.32%
water jug: 0.21%
ashcan: 0.14%
bottlecap: 0.12%
